# Row Level security - princip činnosti

## Přípravné kroky
- Založení databáze
- Založení uživatelů a jejich oprávnění 
- Založení jednoduché tabulky
- Vložení testovacích záznamů

In [ ]:
use master
go

if db_id('RLS') is not NULL
    alter database RLS set single_user with rollback immediate
GO

drop database if exists RLS
go

create database RLS
go

use RLS
go

create role CommonUsers
go

create user Adam without login
create user Bob without login
create user BigBoss without login
go

alter role CommonUsers add member Adam
alter role CommonUsers add member Bob
alter role CommonUsers add member BigBoss
go

grant select, insert, update, delete on schema::dbo to CommonUsers
go

create table dbo.SimpleRls
(
    Id int not null identity constraint pk_SimpleRls primary key
    , Jmeno nvarchar(30) not null
    , Bohatstvi int not null
    , Uzivatel nvarchar(30) not null
)
go

insert dbo.SimpleRls values
('Jiří', 1000, 'Adam')
, ('Jakub', 200, 'Adam')
, ('Vladimír', 1100, 'Adam')
, ('Jana', 1000, 'Bob')
go

Jednoduchý test, že každý z uživatelů může do tabulky a vidí všechny záznamy.

In [ ]:
use RLS
go

execute as user = 'Bob'
select * from dbo.SimpleRls
revert

## Co je cílem RLS
S použitím obyčejného dotazu zavedeme omezení viditelnosti záznamů v tabulce podle uživatele

In [ ]:
use RLS
go

select USER_NAME()
select * from dbo.SimpleRls where Uzivatel = USER_NAME()
go

Výsledek dotazu je prázdný, protože v žádném záznamu není uživatel **dbo**. Pro tuto chvíli musíme rozšířit podmínku, abychom "vypnuli" filtr pro uživatele s oprávněním na celou tabulku.

In [ ]:
use RLS
go

select USER_NAME()
select * from dbo.SimpleRls where Uzivatel = USER_NAME() or USER_NAME() = 'dbo'
go

**Dbo** není ten uživatel, kterého chceme pustit na celý obsah tabulky, ale my stejnou podmínku můžeme použít na kteréhokoliv uživatele. V našem případě je to uživatel **BigBoss**.

In [ ]:
use RLS
go

execute as user = 'BigBoss'
-- execute as user = 'Bob'
-- execute as user = 'Adam'
select USER_NAME()
select * from dbo.SimpleRls where Uzivatel = USER_NAME() or USER_NAME() = 'BigBoss'
revert
go

Předchozí příkaz ukazuje, jak má filtrování fungovat pro jednotlivé uživatele. Nevýhodou je, že takovou podmínku musí někdo (aplikace) vždy napsat. Proto chceme celý mechanismus udělat univerzální.